In [3]:
pip install pgmpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.9 MB/s eta 0:00:00


In [7]:
!pip install scikit-fuzzy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 17.0 MB/s eta 0:00:00


In [11]:
!pip install paho-mqtt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 3.6 MB/s eta 0:00:00


In [16]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Definir estructura de la red
model = BayesianNetwork([
    ('Humedad', 'EstadoCultivo'),
    ('Temperatura', 'EstadoCultivo'),
    ('Fertilidad', 'EstadoCultivo')
])

# Definir distribuciones de probabilidad condicional
cpd_humedad = TabularCPD(variable='Humedad', variable_card=3, values=[[0.2], [0.5], [0.3]])
cpd_temperatura = TabularCPD(variable='Temperatura', variable_card=3, values=[[0.3], [0.4], [0.3]])
cpd_fertilidad = TabularCPD(variable='Fertilidad', variable_card=2, values=[[0.7], [0.3]])

# CPD del estado del cultivo basado en humedad, temperatura y fertilidad
cpd_estado = TabularCPD(
    variable='EstadoCultivo', variable_card=2,
    values=[
        [0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.7, 0.6, 0.5, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.5, 0.4, 0.3],
        [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.3, 0.4, 0.5, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.5, 0.6, 0.7]
    ],
    evidence=['Humedad', 'Temperatura', 'Fertilidad'],
    evidence_card=[3, 3, 2]
)

# Añadir los CPDs al modelo
model.add_cpds(cpd_humedad, cpd_temperatura, cpd_fertilidad, cpd_estado)
model.check_model()

# Inferencia
inference = VariableElimination(model)
estado_predicho = inference.map_query(variables=['EstadoCultivo'], evidence={'Humedad': 2, 'Temperatura': 1, 'Fertilidad': 0})
print(f"Estado del cultivo predicho: {estado_predicho}")


0it [00:00, ?it/s]

0it [00:00, ?it/s]

Estado del cultivo predicho: {'EstadoCultivo': 1}


In [17]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# Definición de variables difusas
humedad = ctrl.Antecedent(np.arange(0, 101, 1), 'humedad')
temperatura = ctrl.Antecedent(np.arange(0, 41, 1), 'temperatura')
riego = ctrl.Consequent(np.arange(0, 101, 1), 'riego')

# Definir membresías
humedad['baja'] = fuzz.trimf(humedad.universe, [0, 0, 50])
humedad['media'] = fuzz.trimf(humedad.universe, [25, 50, 75])
humedad['alta'] = fuzz.trimf(humedad.universe, [50, 100, 100])

temperatura['fria'] = fuzz.trimf(temperatura.universe, [0, 0, 20])
temperatura['ideal'] = fuzz.trimf(temperatura.universe, [10, 20, 30])
temperatura['calurosa'] = fuzz.trimf(temperatura.universe, [20, 40, 40])

riego['bajo'] = fuzz.trimf(riego.universe, [0, 0, 50])
riego['medio'] = fuzz.trimf(riego.universe, [25, 50, 75])
riego['alto'] = fuzz.trimf(riego.universe, [50, 100, 100])

# Definición de reglas difusas
rule1 = ctrl.Rule(humedad['baja'] & temperatura['calurosa'], riego['alto'])
rule2 = ctrl.Rule(humedad['media'] & temperatura['ideal'], riego['medio'])
rule3 = ctrl.Rule(humedad['alta'] | temperatura['fria'], riego['bajo'])

# Sistema de control
riego_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])
riego_simulador = ctrl.ControlSystemSimulation(riego_ctrl)

# Entrada de valores para las variables
riego_simulador.input['humedad'] = 30
riego_simulador.input['temperatura'] = 25

# Computar resultado difuso
riego_simulador.compute()
print(f"Nivel de riego recomendado: {riego_simulador.output['riego']}")


Nivel de riego recomendado: 65.72560202788338


In [18]:
import paho.mqtt.client as mqtt

# Configuración de MQTT
broker = "test.mosquitto.org"  # Broker público
puerto = 1883
topico_publicar = "colab/test"
topico_suscribir = "colab/test"

# Funciones de conexión y recepción
def on_connect(client, userdata, flags, rc):
    print(f"Conectado al broker MQTT con código de retorno {rc}")
    client.subscribe(topico_suscribir)

def on_message(client, userdata, msg):
    print(f"Mensaje recibido: {msg.topic} {msg.payload.decode()}")

# Configuración del cliente MQTT
cliente = mqtt.Client()
cliente.on_connect = on_connect
cliente.on_message = on_message
cliente.connect(broker, puerto, 60)

# Publicar un mensaje
cliente.loop_start()
cliente.publish(topico_publicar, "Hola desde Google Colab")
cliente.loop_stop()


<ipython-input-18-d20067db0be1>:18: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  cliente = mqtt.Client()


<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>